In [2]:
from utils import bsh
import time
from datetime import datetime as dt
import sys
import traceback
import numpy as np

In [2]:
def cmdTraceback(e):
    eType, eValue, eTraceback = sys.exc_info()
    tbList = traceback.extract_tb(eTraceback)
    where = tbList[-1]

    return "%r at %s:%d" % (eValue, where[0], where[1])

In [3]:
def createSimData():
    sim = bsh('simulation', 'toto', 5432)
    sim.startComm()

    simData = {}

    for cmdStr in sim.cmdList:
        res = []

        for state in sim.states:
            sim.sendOneCommand('init')
            sim.sendOneCommand(state)
            stat = sim.sendOneCommand('status')

            ret = sim.sendOneCommand(cmdStr)
            newstat = int(sim.sendOneCommand('status'))
            res.append((state, (ret, newstat)))

        simData[cmdStr] = dict(res)

    return simData

In [4]:
def testBshState(simData, mode, host, port, ts=2):
    logger.info('==== STARTING BSH FUNCTIONAL TEST ====')
   
    ope = bsh(mode, host, port)
    ope.startComm()
    logger.info('==== BSH BOARD in mode %s  host : %s  port: %d ====\r\n'%(mode, host, port))
    for cmdStr in ope.cmdList:
        logger.info('==== TESTING CMD %s ====' % cmdStr)
        for state in ope.states:
            try: 
                ope.sendOneCommand('init')
                time.sleep(ts)
                ope.sendOneCommand(state)

                status = int(ope.sendOneCommand('status'))
                if status != ope.statList[state]:
                    logger.error('warning status (%d) should be %d' % (status, ope.statList[state]))

                statword = int(ope.sendOneCommand('statword'))
                if statword != ope.statwords[state]:
                    logger.error('warning statword (%d) should be %d' % (statword, ope.statwords[state]))

                logger.info('testing on state %s ( status=%d )' % (state, status))
                time.sleep(ts)
                ret = ope.sendOneCommand(cmdStr)

                newstat = int(ope.sendOneCommand('status'))

                if simData[cmdStr][state] != (ret, newstat):
                    logger.error('FAILED !!! ret : %sok, stat:%d'%(ret, newstat))
                else:
                    logger.info('OK ret : %sok, stat:%d'%(ret, newstat))
                    
                time.sleep(ts)
            except Exception as e:
                logger.error(cmdTraceback(e))
            finally:
                ope.closeSock()
                
        logger.info('==== CMD %s VALIDATED ====\r\n' % cmdStr)
              

In [5]:
def testBiaConfig(mode, host, port, ts=2):
    testConf = [dict(biaStrobe='off', biaDuty=255, biaPeriod=5000),
                dict(biaStrobe=None, biaDuty=200, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=100, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=50, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=45, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=40, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=35, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=30, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=25, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=20, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=15, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=10, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=5, biaPeriod=None),
                dict(biaStrobe='on', biaDuty=255, biaPeriod=50),
                dict(biaStrobe=None, biaDuty=200, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=150, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=100, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=50, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=20, biaPeriod=None),
                dict(biaStrobe=None, biaDuty=255, biaPeriod=200),
                dict(biaStrobe=None, biaDuty=None, biaPeriod=500),
                dict(biaStrobe=None, biaDuty=None, biaPeriod=1000),
                dict(biaStrobe=None, biaDuty=None, biaPeriod=2000),
                dict(biaStrobe='off', biaDuty=255, biaPeriod=5000),
               ]
    
    logger.info('==== STARTING BSH CONFIG TEST ====')

    ope = bsh(mode, host, port)
    ope.startComm()
    logger.info('==== BSH BOARD in mode %s  host : %s  port: %d ====\r\n'%(mode, host, port))

    state = 'bia_on'
    ope.sendOneCommand('init')
    time.sleep(ts)
    ope.sendOneCommand(state)

    status = int(ope.sendOneCommand('status'))
    if status != ope.statList[state]:
        logger.error('warning status (%d) should be %d' % (status, ope.statList[state]))

    statword = int(ope.sendOneCommand('statword'))
    if statword != ope.statwords[state]:
        logger.error('warning statword (%d) should be %d' % (statword, ope.statwords[state]))
        
    for kwargs in testConf:
        try: 
            logger.info('==== TESTING CONF %s ===='%kwargs)
            ope.setBiaConfig(**kwargs)
            strobe, period, duty = ope.getBiaConfig()
            
            biaStrobe = kwargs['biaStrobe']
            biaDuty = kwargs['biaDuty']
            biaPeriod = kwargs['biaPeriod']
            
            if biaStrobe is not None and biaStrobe != strobe:
                raise ValueError('bia strobe is not correct')

            if biaPeriod is not None and biaPeriod != period:
                raise ValueError('bia period is not correct')

            if biaDuty is not None and biaDuty != duty:
                raise ValueError('bia duty is not correct')
            
            time.sleep(10)
            photo = []
            for i in range(10):
                __, phr = ope.sendOneCommand('read_phr').split(',')
                photo.append(int(phr))
            photo = np.array(photo)
            
            logger.info('==== Photores=%d+-%.2f ===='%(np.median(photo), np.std(photo)))
            time.sleep(ts)
            logger.info('==== TEST CONF OK ==== \r\n'%kwargs)
            
        except Exception as e:
            logger.error(cmdTraceback(e))
        
        finally:
             ope.closeSock()
          
    ope.sendOneCommand('bia_off')

In [6]:
def testShutterTiming(mode, host, port, ts=5, nb=20):
    
   
    logger.info('==== STARTING SHUTTERS TIMING TEST ====')

    ope = bsh(mode, host, port)
    ope.startComm()
    logger.info('==== BSH BOARD in mode %s  host : %s  port: %d ====\r\n'%(mode, host, port))

    for shut in ['shut', 'blue', 'red']:
        state = '%s_open'%shut
        for i in range(nb):
            try:
                logger.info('==== TEST NB %d/%d ===='%(i+1, nb))
                ope.sendOneCommand('init')
                time.sleep(2)

                logger.info('==== SENDING %s ===='%state)
                ope.sendOneCommand(state)
                logger.info('==== RETURNING %s ===='%state)

                status = int(ope.sendOneCommand('status'))
                if status != ope.statList[state]:
                    logger.error('warning status (%d) should be %d' % (status, ope.statList[state]))

                statword = int(ope.sendOneCommand('statword'))
                if statword != ope.statwords[state]:
                    logger.error('warning statword (%d) should be %d' % (statword, ope.statwords[state]))

                time.sleep(ts)

                logger.info('==== SENDING %s_close ===='%shut)
                ope.sendOneCommand('%s_close'%shut)
                logger.info('==== RETURNING %s_close ===='%shut)

                status = int(ope.sendOneCommand('status'))
                if status != ope.statList['init']:
                    logger.error('warning status (%d) should be %d' % (status, ope.statList['init']))

                statword = int(ope.sendOneCommand('statword'))
                if statword != ope.statwords['init']:
                    logger.error('warning statword (%d) should be %d' % (statword, ope.statwords['init']))

                time.sleep(ts)
               
            except Exception as e:
                logger.error(cmdTraceback(e))
            
            finally:
                 ope.closeSock()          

In [7]:
simData = createSimData()

In [13]:
import logging

#logFolder ='/software/ait/logs/bsh'
logFolder ='/home/arnaud/AIT-PFS/bshlogs'
logFile = '%s/%s-bshBoard-funcTest.log' % (logFolder, dt.utcnow().strftime('%Y-%m-%d'))

logger = logging.getLogger('bshBoard')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler(logFile)
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

In [9]:
host = 'arduitest.pfs'
port = 23
mode = 'operation'

In [10]:
testBshState(simData, mode=mode, host=host, port=port)

In [12]:
testBiaConfig(mode=mode, host=host, port=port)

2019-03-07 16:25:35,413 - bshBoard - ERROR - gaierror(-2, 'Name or service not known') at /home/arnaud/mhs/devel/ics_enuActor/engineering/biasha/utils.py:25
2019-03-07 16:25:35,413 - bshBoard - ERROR - gaierror(-2, 'Name or service not known') at /home/arnaud/mhs/devel/ics_enuActor/engineering/biasha/utils.py:25


In [14]:
testShutterTiming(mode=mode, host=host, port=port)

2019-03-07 16:33:53,615 - bshBoard - ERROR - OSError('Timed out reading character from bshIO') at /home/arnaud/mhs/devel/ics_enuActor/engineering/biasha/utils.py:110
2019-03-07 16:33:53,615 - bshBoard - ERROR - OSError('Timed out reading character from bshIO') at /home/arnaud/mhs/devel/ics_enuActor/engineering/biasha/utils.py:110
2019-03-07 16:33:53,615 - bshBoard - ERROR - OSError('Timed out reading character from bshIO') at /home/arnaud/mhs/devel/ics_enuActor/engineering/biasha/utils.py:110
